# Travail 1

### Dans le cadre du cours IFT3700 - Science des données

Effectué par <br> Alexandre Chartrand,<br> Lauranne Deaudelin, <br> Amélie Lacombe Robillard, et <br> Stéphane Verville-Vohl

# TODO

- Binary Partition -> Plugger la matrice de dissimilarité
- ISOMAP -> Trouver un moyen de calculer les bon eigenvectors avec la distance custom
- Mettre des scores à chacun des 5 algorithmes pour pouvoir comparer

- Doubler tout, pour faire Euclédian (en cours)


## Libraires à installer 
Il faut s'arrurer que la librairie pyclustering est installée.  
https://pypi.org/project/pyclustering/0.8.0/

## Import libraries

In [9]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
import csv
import time
%matplotlib inline

## Import data

In [10]:
TRAINING_SIZE = 1000
TEST_SIZE = 25

def extract_data(dataset, size):
    data = open(dataset)
    reader = csv.reader(data)
    data_points = []
    count = 0
    for row in reader:
        if count == 0:
            count += 1
            continue
        elif count <= size:
            data_points.append(row)
            count += 1
        else:
            break
    data.close()

    # Convert data in numpy array, with int type
    ndarray_dataset = np.array(data_points).astype(int)
    
    # Data input
    inputs = ndarray_dataset[:,1:]
    # Target vector (label vector)
    targets = ndarray_dataset[:,0]
    
    return (inputs, targets)

training_data = extract_data('mnist_train.csv', TRAINING_SIZE)
training_inputs = training_data[0]
training_targets = training_data[1]

test_data = extract_data('mnist_test.csv', TEST_SIZE)
test_inputs = test_data[0]
test_targets = test_data[1]

## Custom Similarity Measure

In [11]:
def custom_similarity_measure(x,y):
    return min_translation_distance(x,y)
    #return np.sum(abs(x-y))

In [12]:
from scipy.spatial import distance as dist
#### Helper functions ######

# Crop a matrix according to margins of format (top, bottom, left, right)
def crop_matrix(x, margins):
    return x[margins[0]:x.shape[0]-margins[1],margins[2]:x.shape[1]-margins[3]]

# Pad a matrix with zeros according to a margin of format (top, bottom, left, right)
def pad_matrix(x, margins):
    x_tilda = np.zeros((x.shape[0]+margins[0]+margins[1], x.shape[1]+margins[2]+margins[3]), dtype=int)
    for i in range(0, x.shape[0]):
        for j in range(0, x.shape[1]) :
            x_tilda[i+margins[0],j+margins[2]] = x[i,j]
    return x_tilda

# TO REMOVE
def euclidean_distance(x,y) :
    return dist.dice(x, y)

##### Custom similarity measure based on smallest translation distance
#Return the smallest translation distance of all possible translations of a number of pixels d or smaller
# in all directions (top, bottom, left, right and diagonals) 
def min_translation_distance(x, y, d=1):
 
    #This should be a hyper parameter but it would change the function signature
    #d = 1
    
    transform_margins = []
    for i in range(1, d+1) :
        transform_margins.extend([[d-i,d+i,d,d], #top
                                  [d+i, d-i, d,d], #bottom
                                  [d, d, d-i, d+i], #left 
                                  [d, d, d+i, d-i], #right
                                  [d-i, d+i, d-i, d+i], #top-left
                                  [d-i, d+i, d+i, d-i], #top-right
                                  [d+i, d-i, d-i, d+i], #bottom-left
                                  [d+i, d-i, d+i, d-i]]) #bottom-right
           

    
    #We assume that we are working with mnist data, or else reshaping dimensions are not valid
    if (x.size != 784) or (x.size != y.size) :
        print("invalid x or y")

    #Reshape into matrix for ease 
    #y_matrix = y.reshape(28,28)
    
    #Test without translation : 
    #smallest_dist = euclidean_distance(x.reshape(28,28), y_matrix)
    smallest_dist = euclidean_distance(x , y)

    
    #We pad x with white pixels all around. 
    #This ensure that we will always be working with same size matrix,
    # and won't affect the overall distance (as the distance between two white pixels will be 0 and won't affect the total distance)
    x_padded = pad_matrix(x.reshape(28,28), [d,d,d,d])

    #Test all translations and keep the one with the smallest distance
    for i in range(0, len(transform_margins)) :
        #distance = euclidean_distance(crop_matrix(x_padded, transform_margins[i]), y_matrix)
        distance = euclidean_distance(crop_matrix(x_padded, transform_margins[i]).reshape(1, 784), y)
        
        if(distance < smallest_dist) :
            smallest_dist = distance
        
    return smallest_dist

## Euclidean Distance

In [13]:
def euclidean_similarity_measure(x, y):
    return np.sqrt(np.dot(x,x) - 2 * np.dot(x,y) + np.dot(y,y))

## Dissimilarity Matrix with the custom measure

In [14]:
def generate_distance_matrix(data, distance_type):
    nb_examples = data.shape[0]
    distance_ndarray = np.zeros((nb_examples,nb_examples))
    
    for i in range(nb_examples):
        for j in range(nb_examples):
            if distance_type is 'euclidean':
                distance_ndarray[i][j] = euclidean_similarity_measure(data[i],data[j])
            elif distance_type is 'custom':
                distance_ndarray[i][j] = custom_similarity_measure(data[i],data[j])
            else:
                print("accepted values for argument 'distance_type': 'euclidean','custom'")
                return
    return distance_ndarray

## Compute dissimilarity matrices on the data

In [15]:
start_time = time.time()
custom_distance_matrix = generate_distance_matrix(training_inputs, 'custom')
print("--- %s seconds ---" % (time.time() - start_time))
euclidean_distance_matrix = generate_distance_matrix(training_inputs, 'euclidean')

--- 919.5779030323029 seconds ---


## K-medoid Algorithm

In [16]:
start_time = time.time()
from sklearn import metrics
from pyclustering.cluster.kmedoids import kmedoids

K = 10

# Initialization of medoids
initial_medoids = [i for i in range(K)]

def map_medoid_labels(medoids):
    medoid_labels = []
    for i,medoid in enumerate(medoids):
        medoid_labels.append((i, training_targets[medoid]))
    return medoid_labels
    
    
def kmedoid_predict(medoids, distance_type):
    predictions = []
    # For all test inputs
    for row in test_inputs:
        min_arg = (404, float('inf'))
        for medoid in medoids:
            # Find the closest medoid
            if distance_type is 'euclidean':
                candidate_distance = euclidean_similarity_measure(row, training_inputs[medoid])
            elif distance_type is 'custom':
                candidate_distance = custom_similarity_measure(row, training_inputs[medoid])
            if candidate_distance < min_arg[1]:
                min_arg = (medoid, candidate_distance)
        # Return the nearest medoid's label
        predictions.append(training_targets[min_arg[0]])
        ndarray_predictions = np.array(predictions).astype(int)
    return ndarray_predictions


def kmedoid_prediction_score(predictions):
    error = 0
    for i, prediction in enumerate(predictions):
        target = test_targets[i]
        if prediction != target:
            error += 1
    return 1 - error/predictions.shape[0]


# ================ Euclidean k-Medoid ======================
euclidean_kmedoids = kmedoids(euclidean_distance_matrix, initial_medoids, data_type='distance_matrix')
euclidean_kmedoids.process()
euclidean_clusters = euclidean_kmedoids.get_clusters()
euclidean_medoids = euclidean_kmedoids.get_medoids()
euclidean_predictions = kmedoid_predict(euclidean_medoids, 'euclidean')
euclidean_kmedoid_score = kmedoid_prediction_score(euclidean_predictions)

print('\nEuclidean distance results:')
print("\nTraining dataset size:", TRAINING_SIZE)
print("\nNumber of medoids:", K)
print("Cluster mapping:", map_medoid_labels(euclidean_medoids))
print("Euclidean prediction score:", euclidean_kmedoid_score)
print("Euclidean homogeneity:", metrics.homogeneity_score(test_targets, euclidean_predictions)  )
print("Euclidean cluster completeness:", metrics.completeness_score(test_targets, euclidean_predictions))
                             
# ==================== Custom k-Medoid =====================
custom_kmedoids = kmedoids(custom_distance_matrix, initial_medoids, data_type='distance_matrix')
custom_kmedoids.process()
custom_clusters = custom_kmedoids.get_clusters()
custom_medoids = custom_kmedoids.get_medoids()                             
custom_predictions = kmedoid_predict(custom_medoids, 'custom')
custom_kmedoid_score = kmedoid_prediction_score(custom_predictions)

print('\nCustom distance results:')
print("\nTraining dataset size:", TRAINING_SIZE)
print("\nNumber of medoids:", K)
print("Cluster mapping:", map_medoid_labels(custom_medoids))
print("Custom prediction score:", custom_kmedoid_score)
print("Custom homogeneity:", metrics.homogeneity_score(test_targets, custom_predictions))
print("Custom cluster completeness:", metrics.completeness_score(test_targets, custom_predictions))



Euclidean distance results:

Training dataset size: 1000

Number of medoids: 10
Cluster mapping: [(0, 5), (1, 0), (2, 9), (3, 1), (4, 9), (5, 2), (6, 1), (7, 3), (8, 1), (9, 9)]
Euclidean prediction score: 0.48
Euclidean homogeneity: 0.4203948192946918
Euclidean cluster completeness: 0.8085989114317371

Custom distance results:

Training dataset size: 1000

Number of medoids: 10
Cluster mapping: [(0, 5), (1, 0), (2, 4), (3, 1), (4, 9), (5, 0), (6, 1), (7, 3), (8, 1), (9, 4)]
Custom prediction score: 0.43999999999999995
Custom homogeneity: 0.5898269368309494
Custom cluster completeness: 0.7560253108406848


In [17]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.8936104774475098 seconds ---


## Binary partition Algorithm

In [ ]:
from sklearn import tree
binary_partition = tree.DecisionTreeClassifier()
binary_partition = binary_partition.fit(X, Y)

print(binary_partition.decision_path(X))
print(binary_partition.score(X,Y))
binary_partition.get_params()

# TODO : calculer l'impureté

## PCoA  Algorithm

In [ ]:
from sklearn.manifold import MDS

#PCOA avec euclidien built-in
pcoa_builtin = MDS(n_components=2, dissimilarity='euclidean')
builtin_transformed = pcoa_builtin.fit_transform(training_inputs)

#PCOA avec matrice de dissimilarité euclidienne
pcoa_euclid = MDS(n_components=2, dissimilarity='precomputed')
euclid_transformed = pcoa_euclid.fit_transform(euclidean_distance_matrix)

#PCOA avec matrice de dissimilarité custom
pcoa_custom = MDS(n_components=2, dissimilarity='precomputed')
custom_transformed = pcoa_custom.fit_transform(custom_distance_matrix)

In [ ]:
def extract_data_by_label(inputs, targets, label) :
    return(inputs[np.where(targets[:] == label)[0], :])
    
def plot_graph_by_labels(inputs, targets) :
    colors = ['b', 'r', 'g', 'y', 'c', 'k', 'darkorange', 'lightcoral', 'lime', 'grey']
    for i in range(0, 10) :
            extracted = extract_data_by_label(inputs, targets, i)
            plt.scatter(extracted[:,0], extracted[:,1], 8, colors[i])
    pylab.xlabel('x')
    pylab.ylabel('y')
    plt.title('MNIST dans l\'espace 2D avec PCoA custom')
    plt.show()

plot_graph_by_labels(builtin_transformed, training_targets)

plot_graph_by_labels(euclid_transformed, training_targets)

plot_graph_by_labels(custom_transformed, training_targets)

## ISOMAP Algorithm

In [ ]:
## ISOMAP Algorithm

from sklearn.manifold import Isomap
ISOMAP = Isomap(n_neighbors=1, n_components=2)
ISOMAP.fit(inputs, targets)
ISOMAP.get_params()
print(ISOMAP.dist_matrix_)

# TODO : calculer l'impureté

## K Nearest Neighbors Algorithm

In [18]:
start_time = time.time()
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import BallTree
from sklearn.metrics import accuracy_score

neighbourinos = 5

KNN_euclid = KNeighborsClassifier(n_neighbors=neighbourinos,  metric='euclidean')
KNN_custom = KNeighborsClassifier(n_neighbors=neighbourinos, algorithm='ball_tree', metric=custom_similarity_measure)

KNN_euclid.fit(training_inputs, training_targets)
KNN_custom.fit(training_inputs, training_targets)

score_euclid = accuracy_score(KNN_euclid.predict(test_inputs), test_targets)
score_custom = accuracy_score(KNN_custom.predict(test_inputs), test_targets)

print("Training dataset size:", TRAINING_SIZE)
print("Neighbours taken into account:", neighbourinos)
print("Euclidean score:", score_euclid)
print("Custom score:", score_custom)

Training dataset size: 1000
Neighbours taken into account: 5
Euclidean score: 0.72
Custom score: 0.56


In [19]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 7.02421498298645 seconds ---
